In [ ]:
from tensorflow.keras.losses import MSE
import tensorflow as tf
import keras
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from pathlib import Path

https://www.tensorflow.org/tutorials/generative/adversarial_fgsm

https://pyimagesearch.com/2021/03/01/adversarial-attacks-with-fgsm-fast-gradient-sign-method/

In [ ]:
def generate_image_adversary(model, image, label, eps=1 / 255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tacked for
        # gradient updates
        tape.watch(image)
        # use our model to make predictions on the input image and
        # then compute the loss
        pred = model(image)
        loss = MSE(label, pred)
    # calculate the gradients of loss with respect to the image, then
    # compute the sign of the gradient
    gradient = tape.gradient(loss, image)
    signedGrad = tf.sign(gradient)
    # construct the image adversary
    adversary = (image + (signedGrad * eps)).numpy()
    # return the image adversary to the calling function
    return adversary

In [ ]:
def transform_model_image(model, image):
    model = keras.models.load_model(model)

    img = cv2.imread(image)
    img = np.reshape(img, [1, 224, 224, 3])

    return model, img

In [ ]:
def make_prediction(model, image):
    predicted_values = model.predict(image)
    predicted_class = np.array(predicted_values).argmax()
    return predicted_class

In [ ]:
path = Path("test_data_smaller")
adversarial_images_dir = Path(path/'images/adversarial_images/')

if not os.path.exists(adversarial_images_dir):
    os.makedirs(adversarial_images_dir)

In [ ]:
images_dir = Path(path/'images/images')

painting_list = []
for path, subdirs, files in os.walk(images_dir):
    for name in files:
        img = os.path.join(path, name)
        painting_list.extend([img])
print(painting_list)

In [ ]:
for painting in painting_list:
    model, image = transform_model_image("model_keras_test_smaller.h5", painting)
    label = make_prediction(model, image)
    adversary = generate_image_adversary(model, image, label)
    adversary_label = make_prediction(model, adversary)
    i = 1
    while adversary_label == label and i <= 10:
        i = i + 2
        adversary = generate_image_adversary(model, image, label, i)
        adversary_label = make_prediction(model, adversary)
    filename = os.path.basename(painting)
    
    predicted_values_image = model.predict(image)
    predicted_class_image = np.array(predicted_values_image).argmax()

    predicted_values_adversary = model.predict(adversary)
    predicted_class_adversary = np.array(predicted_values_adversary).argmax()
    if predicted_class_image != predicted_class_adversary:
        cv2.imwrite(str(adversarial_images_dir)+'/'+filename, adversary[0])
